In [87]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os


%matplotlib inline


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    

    #     print("norma Image")
    # plt.imshow(image)

    # plt.show()

    gray = grayscale(image)
    # print("gray Image")
    # plt.imshow(gray,cmap="gray")
    # plt.show()


    blur_gray = gaussian_blur(image, 3)
    #     print("blur_gray")
    #     plt.imshow(blur_gray)
    #     plt.show()

    edges = canny(blur_gray, 50, 150)
    # print("edges")
    # print(edges.size)
    # plt.imshow(edges,cmap="gray")
    # plt.show()

    imshape = image.shape
    vertices = np.array([[(90, imshape[0]), (450, 320), (550, 330
                                                         ), (imshape[1], imshape[0])]], dtype=np.int32)

    masked_edges = region_of_interest(edges, vertices)

    # print("region")
    # print(masked_edges.size)
    # plt.imshow(masked_edges,cmap="gray")
    # plt.show()

    rho = 10
    theta = np.pi / 180
    threshold = 15
    min_line_len = 40
    max_line_gap = 20
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_len=40, max_line_gap=20)

    # print("lines")
    # print(lines.size)
    # plt.imshow(lines)
    #
    # plt.show()

    color_edges = np.dstack((edges, edges, edges))
#     plt.imshow(color_edges)

#     #
#     plt.show()

    line_edge = weighted_img(image, lines, λ=10)
    # print("line_edges.size",line_edge.size)

#     plt.imshow(line_edge)
#     plt.show()
    return line_edge

dir=os.listdir("test_videos")
v=[]


for videos in dir:
    print(videos)
    v.append(videos)
print(v)
print(v[0])
print(v[1])
print(v[2])
    
white_output = 'outputV/'+v[1]
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
# clip1 = VideoFileClip("test_videos/"+v[1])
clip1 = VideoFileClip("test_videos/"+v[1])
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
<source src="{0}">
</video>
""".format(white_output))

challenge.mp4
solidWhiteRight.mp4
solidYellowLeft.mp4
['challenge.mp4', 'solidWhiteRight.mp4', 'solidYellowLeft.mp4']
challenge.mp4
solidWhiteRight.mp4
solidYellowLeft.mp4
[MoviePy] >>>> Building video outputV/solidWhiteRight.mp4
[MoviePy] Writing video outputV/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:08<00:00, 24.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputV/solidWhiteRight.mp4 

CPU times: user 21.1 s, sys: 700 ms, total: 21.8 s
Wall time: 9.33 s
